In [77]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict,Union
load_dotenv(override=True)

True

#### Pydantic

In [117]:
from pydantic import BaseModel,Field
        
class schedule_details(BaseModel):
    contact: str
    "Contact Number of the User"
    
    schedule_date_from: str
    "Schedule data from in format 'YYYY-MM-DD'"
    
    schedule_date_to: str
    "Schedule data to in format 'YYYY-MM-DD'"

class NLP2SQLOutput(BaseModel):
    sql_query: str
    "Generated SQL Query based on user's natural language request."

    def __iter__(self):
        # Allow unpacking or list conversion, e.g., list(obj) or obj1, obj2 = obj
        yield self.sql_query

class IntentChecker(BaseModel):
    """
    Model to determine which agent should handle a given request.
    """
    SQL_agent: bool = Field(
        ...,
        description="Set to True if SQL Agent is required for querying schedules or existing information of athletes or coaches."
    )
    knowledge_agent: bool = Field(
        ...,
        description="Set to True if the Knowledge Agent is required to handle general information queries."
    )

class KnowledgeAgentOutput(BaseModel):
    """
    Knowledge Agent's response in a way that the Front Desk Agent can understand.
    """
    response_to_frontdesk_agent: str
    
class FrontDeskOutput(BaseModel):
    """
    Model to Respond to the end user
    """
    respond_to_user : Union[str, bool]
    "A response the End user will have from the Call center Agent"
    ask_the_agent : Union[str, bool]
    "This the the prompt with detailes emphased to the intent detection Agent"


### Agents Declaration

In [83]:
intent_agent = Agent(
    name="Intent Judge",
    instructions=(
        "You are an intent classifier. Based on the user's message, decide which functional agent should handle it.\n"
        "- Set `SQL_agent` to True **if the message includes queries about schedules, training sessions, matches, or anything involving database lookups** for athletes or coaches.\n"
        "- Set `knowledge_agent` to True **if the message includes informational, instructional, or explanatory questions**, such as rules, guidelines, or general knowledge.\n"
        "Return a JSON with True or False for both `SQL_agent` and `knowledge_agent`. If the message includes both types of intent, set both to True."
    ),
    model="gpt-4o-mini",
    output_type=IntentChecker
)


In [86]:
with trace("Intent Detection"):
    result = await Runner.run(intent_agent, "Hi, what are the schedule for today and instructions any to attend")

In [87]:
print(result)

RunResult:
- Last agent: Agent(name="Intent Judge", ...)
- Final output (IntentChecker):
    {
      "SQL_agent": true,
      "knowledge_agent": true
    }
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [88]:
with trace("Intent Detection"):
    result = await Runner.run(intent_agent, "Hi, what are the schedule for today")

In [89]:
print(result)

RunResult:
- Last agent: Agent(name="Intent Judge", ...)
- Final output (IntentChecker):
    {
      "SQL_agent": true,
      "knowledge_agent": false
    }
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [90]:
# sql_agent
sql_agent = Agent(
    name="SQL Agent",
    instructions=(
        "Convert the user's natural language query into a valid SQL query "
        "to retrieve or update athlete/coach schedule information. "
        """Respond only with a valid SQL query string in the 'sql_query' field.
        You are an SQL Architect and you are tasked to generate Postgress SQL based on natural langauge question and schema below: 
        ### Schema: 
            CREATE TABLE public.athlete_batches (
                id serial4 NOT NULL,
                contact_no varchar(15) NOT NULL,
                athlete_name text NOT NULL,
                batch_name text NOT NULL,
                "date" date NOT NULL,
                start_time time NOT NULL,
                end_time time NOT NULL,
                status varchar(10) NOT NULL,
                CONSTRAINT athlete_batches_pkey PRIMARY KEY (id),
                CONSTRAINT athlete_batches_status_check CHECK (((status)::text = ANY ((ARRAY['Open'::character varying, 'Cancelled'::character varying])::text[])))""" 
    ),
    model="gpt-4o-mini",
    output_type=NLP2SQLOutput
)

In [91]:
async def call_agent(agent, user_query: str):
    with trace(f"{agent.name}"):
        result = await Runner.run(agent, user_query)
    return result.final_output

In [97]:
await call_agent(sql_agent,"Hi, what are the schedule for today")

NLP2SQLOutput(sql_query='SELECT * FROM public.athlete_batches WHERE date = CURRENT_DATE;')

In [98]:
# knowledge_agent
knowledge_agent = Agent(
    name="Knowledge Agent",
    instructions=(
        "You are an expert on sports rules, strategies, and official sports news only. "
        "Answer **only** questions related to sports rules, game strategies, "
        "or recent verified sports news. "
        "Do not answer questions about controversies, politics, personal opinions, or any unrelated topics. "
        "If asked about topics outside this scope, respond politely stating you can only provide information about sports rules, strategies, or official sports news."
    ),
    model="gpt-4o-mini",
    output_type=KnowledgeAgentOutput
)

In [99]:
await call_agent(knowledge_agent,"Hi, what are the schedule for today")

KnowledgeAgentOutput(response_to_frontdesk_agent="I can only provide information about sports rules, strategies, or verified sports news. For schedules, please refer to the official website or app for the specific sports or events you're interested in.")

In [100]:
# frontdesk_agent
frontdesk_agent = Agent(
    name="Front Desk Agent",
    instructions=(
        """
You are a friendly and professional Relationship Manager assisting athletes, coaches, and customers. Your role is to understand their needs clearly, provide accurate information, guide them politely, and route their requests to the right specialist agent when needed.

- When you receive a message, first determine if it is from a user or from another agent.
- If the message is from a user:
    - Greet warmly and confirm understanding.
    - Listen carefully to the user's queries about schedules, training, game rules, or general information.
    - If the request is unclear or off-topic, politely ask for clarification or offer help on common topics.
    - Maintain a helpful and respectful tone throughout the conversation.
    - If the user's request requires a specialist, route it to the appropriate agent and inform the user.
- If the message is from another agent:
    - Respond professionally and concisely, providing the requested information or assistance.
    - If you cannot help, politely indicate so and suggest the next best step.
"""
    ),
    model="gpt-4o-mini",
    output_type=FrontDeskOutput
)

In [101]:
await call_agent(frontdesk_agent,"Hi, what are the schedule for today")

FrontDeskOutput(respond_to_user="Hello! Thank you for reaching out. Could you please specify which schedule you are referring to? Is it for training, games, or something else? I'd love to help you out!", ask_the_agent=False)

### Tools

SQL Python Executor

In [102]:
import psycopg2
from psycopg2.extras import RealDictCursor
@function_tool
def get_sql_respo(sql_query: str):
    """Executes SQL query and returns results from athlete schedules."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="Privacy@100",
            port=5432
        )
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute(sql_query)
        results = cursor.fetchall()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        print("Error executing SQL:", e)
        return []


Agent Response Moderator

In [103]:
@function_tool
async def moderator_tool(agent_respo):
    tools=[get_sql_respo]
    #Response Moderator Agent
    moderator_agent = Agent(
        name = "Agents Response Moderator",
        instructions = (
        """You are an Agents' Response Moderator. Your task is to take the agent’s response and rephrase it into a clear, simple, and smooth version that is easy to understand and friendly in tone.""" 
    f"""The response:{agent_respo}\
    - Maintain the original meaning and key details.\
    - Use natural, conversational language.\
    - Make the response concise but complete.\
    - Ensure the tone is polite, approachable, and professional.\
    - Avoid jargon or overly complex phrases.\
    - You should use the tool get_sql_respo to get SQL response the response\
    Your goal is to enhance readability and improve the user’s experience with clear, friendly communication.\
    """
        ),
        tools=tools
        )
    result = await call_agent(moderator_agent,agent_respo)
    return result

In [105]:
# @function_tool
# async def moderator_tool(agent_respo: str):
#     tools = [get_sql_respo]

#     moderator_agent = Agent(
#         name="Agents Response Moderator",
#         instructions=(
#             f"""You are an Agents' Response Moderator. Your task is to rephrase the following response:\n\n
#             {agent_respo}\n\n
#             - Make it clear, polite, and friendly\n
#             - Preserve meaning\n
#             - Improve readability and flow\n
#             - Avoid jargon\n
#             Use get_sql_respo if needed."""
#         ),
#         tools=tools,
#         model="gpt-4o-mini"
#     )

#     result = await call_agent(moderator_agent, agent_respo)
#     return result


In [106]:
await call_agent(frontdesk_agent,"Hi, what are the schedule for today")

FrontDeskOutput(respond_to_user="Hello! I’d be happy to help with today’s schedule. Could you please specify which team's or athlete's schedule you are looking for?", ask_the_agent=False)

In [120]:
from typing import Dict

# Define moderator_agent once globally (outside function) to avoid redefining it every call
moderator_agent = Agent(
    name="Agents Response Moderator",
    instructions=(
        "You are an Agents' Response Moderator. Your task is to take the agent's response and rephrase it into a clear, "
        "simple, and smooth version that is easy to understand and friendly in tone.\n"
        "- Maintain the original meaning and key details.\n"
        "- Use natural, conversational language.\n"
        "- Make the response concise but complete.\n"
        "- Ensure the tone is polite, approachable, and professional.\n"
        "- Avoid jargon or overly complex phrases.\n"
        "Your goal is to enhance readability and improve the user's experience with clear, friendly communication."
    ),
    tools=[get_sql_respo]
)

In [128]:

async def handle_conversation(user_input: str, context: Dict = {}):
    # 1. Detect intent
    intent_result = await call_agent(intent_agent, user_query=user_input)
    is_sql = intent_result.SQL_agent
    is_knowledge = intent_result.knowledge_agent

    responses = []
    final_response = ""

    if not is_sql and not is_knowledge:
        return "I'm not sure how to help with that yet, but I'll note it down.", context

    # 2. SQL agent flow
    if is_sql:
        sql_response = await call_agent(sql_agent, user_query=user_input)
        print("Raw SQL Response:", sql_response.sql_query)

        # Rephrase the SQL agent’s response
        moderated_sql_response = await call_agent(moderator_agent, user_query=sql_response.sql_query)
        print("Moderated SQL Response:", moderated_sql_response)

        # Append and send through front desk
        responses.append(moderated_sql_response)
        final_response = await call_agent(frontdesk_agent, user_query=moderated_sql_response)

    # 3. Knowledge agent flow
    if is_knowledge:
        knowledge_response = await call_agent(knowledge_agent, user_query=user_input)
        responses.append(knowledge_response)
        final_response = await call_agent(frontdesk_agent, user_query=knowledge_response)

    # 4. Store context
    context["last_intent"] = intent_result
    context["last_response"] = final_response

    return final_response, context


In [130]:
context = {}
final_respo  = await handle_conversation("""My contact number is 6655443322
I wan to attend this session can you make the status Open for me
Flexibility Training
Date: May 24, 2025
Time: 8:30 AM - 9:30 AM
Status: Cancelled""", context)

Raw SQL Response: UPDATE public.athlete_batches SET status = 'Open' WHERE contact_no = '6655443322' AND batch_name = 'Flexibility Training' AND "date" = '2025-05-24' AND start_time = '08:30:00' AND end_time = '09:30:00' AND status = 'Cancelled';
Moderated SQL Response: I see that you're looking to update a specific athlete batch record. Here's a brief explanation: 

You'll be setting the status to "Open" for the Flexibility Training session happening on May 24, 2025, from 8:30 AM to 9:30 AM. This change applies to the batch linked with the contact number 6655443322, and the current status should be "Cancelled" before this update takes effect. 

If you need further help or adjustments, feel free to ask!


In [131]:
context = {}
final_respo  = await handle_conversation("""
My contact number is  6655443322
What are my current schedule for 24th may
""", context)


Raw SQL Response: SELECT athlete_name, batch_name, "date", start_time, end_time, status FROM public.athlete_batches WHERE contact_no = '6655443322' AND "date" = '2023-05-24' AND status = 'Open';
Moderated SQL Response: It seems there are no open schedules for the athlete with contact number 6655443322 on May 24, 2023. If you need further assistance, feel free to ask!


In [47]:
from IPython.display import Markdown, display
display(Markdown(final_respo))

d:\agents\.venv\Lib\site-packages\pygments\regexopt.py:78: RuntimeWarning: coroutine 'handle_conversation' was never awaited
  for group in groupby(strings, lambda s: s[0] == first[0])) \


TypeError: Markdown expects text, not ('sql_query=\'SELECT * FROM public.athlete_batches WHERE "date" = CURRENT_DATE;\'', {'last_intent': IntentChecker(SQL_agent=True, knowledge_agent=False), 'last_response': 'sql_query=\'SELECT * FROM public.athlete_batches WHERE "date" = CURRENT_DATE;\''})

In [41]:
context = {}

user_inputs = [
    "Hi, what are the schedules today?",
    "Tell me about the match rules.",
    "Cancel my 5 PM slot.",
    "Can you explain the coach selection criteria?"
]

for user_input in user_inputs:
    response, context = await handle_conversation(user_input, context)
    print("🤖 Agent says:", response)


Starting conversation with input: Hi, what are the schedules today?
Step 1: Detecting intent...
Intent result: SQL=True, Knowledge=False
Step 2: Routing to agents...
Routing to SQL agent...
SQL agent response: sql_query='SELECT * FROM public.athlete_batches WHERE "date" = CURRENT_DATE;'
Step 3: Combining responses...
Combined response: sql_query='SELECT * FROM public.athlete_batches WHERE "date" = CURRENT_DATE;'
Step 4: Checking if moderation needed...
Response is short enough, skipping moderation
Step 5: Updating context...
Context updated
Conversation handling complete
🤖 Agent says: sql_query='SELECT * FROM public.athlete_batches WHERE "date" = CURRENT_DATE;'
Starting conversation with input: Tell me about the match rules.
Step 1: Detecting intent...
Intent result: SQL=False, Knowledge=True
Step 2: Routing to agents...
Routing to knowledge agent...
Knowledge agent response: Could you please specify which sport's match rules you are interested in? Different sports have varied rules go